In this notebook we write the code for sorting transitions timeseries.

## Loading Data

In [195]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm



ModuleNotFoundError: No module named 'tqdm'

In [2]:
raw_data_dir = '/Users/cfn18/Desktop/Double-Well-SR/Test-Transition-Data/'

def nc_test(file_name):
    return file_name[-3:] == '.nc'

def nc_file_list(d):
    nc_list = []
    for file in os.listdir(d):
        if nc_test(file):
            nc_list.append(d + file)
    return nc_list

integration_files = nc_file_list(raw_data_dir)

In [3]:
ds_file = integration_files[0]
ds = xr.open_mfdataset(ds_file)

## Step 1: Tag points in timeseries near fixed points

In [66]:
def _tag_cold_points(ts, ball_size=0.1):
    return -(np.sqrt((ts.x + 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def _tag_hot_points(ts, ball_size=0.1):
    return (np.sqrt((ts.x - 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def symbolic_ts(ts, ball_size=0.1):
    """
    Identifies all points in a timeseries with -1, 0 or 1.
    -1 means you're close to (-1, 0)
    1 means you're close to (1, 0)
    0 is rest of points.
    """
    cold_points = _tag_cold_points(ts, ball_size)
    hot_points = _tag_hot_points(ts, ball_size)
    return cold_points + hot_points

In [106]:
# Functions take a single timeseries and return transition from hot to cold or vice versa

def cold_to_hot_transitions(ts, ball_size=0.1):
    c2h = [] # List of transitions
    symbolic_path = symbolic_ts(ts, ball_size)
    
    # Identifying sequences of the form (-1, 0, ..., 0, 1)
    for i in np.where(symbolic_path == -1)[0]:
        next_hot_point = np.argmax(symbolic_path[i:])
        
        # Is it only 0's between the -1 and 1?
        if (np.sum(symbolic_path[slice(i, i + next_hot_point + 1)]) == 0): 
            transition = ts.isel(time=slice(i, i + next_hot_point + 1))
            c2h.append(transition)
    return c2h

def hot_to_cold_transitions(ts, ball_size=0.1):
    h2c = [] # List of transitions
    symbolic_path = symbolic_ts(ts, ball_size)
    
    # Identifying sequences of the form (1, 0, ..., 0, -1)
    for i in np.where(symbolic_path == 1)[0]:
        next_cold_point = np.argmin(symbolic_path[i:])
        
        # Is it only 0's between the 1 and -1?
        if (np.sum(symbolic_path[slice(i, i + next_cold_point + 1)]) == 0): 
            transition = ts.isel(time=slice(i, i + next_cold_point + 1))
            h2c.append(transition)
    return h2c

In [204]:
# TO DO: 
# Script opening of all datasets
# Save all transitions in appropriate places
# Arrange this to run in a cluster

c2h = []
h2c = []
for r in ds.realisation:
    ts = ds.sel(realisation=r).drop('realisation')
    c2h += cold_to_hot_transitions(ts, ball_size)
    h2c += hot_to_cold_transitions(ts, ball_size)

In [209]:
times = []
for ts in c2h:
    time = ts.time.max() - ts.time.min()
    times.append(time)

In [ ]:
plt.hist(times, density=True)

(array([[0.        , 0.        , 0.67114094, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.67114094, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.67114094, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.67114094, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.67114094, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.67114094, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 array([ 0.16,  1.65,  3.14,  4.63,  6.12,  7.61,  9.1 , 10.59, 12.08,
        13.57, 15.06]),
 <a list of 14708 BarContainer objects>)